In [97]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime as dt
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from scipy.stats import gamma

File con per ogni riga una provincia, e in colonna il < k > per ogni giorno. Questo dato è indicativo su quanti contatti una persona in media ha in ogni regione. Potrei anche verificare l'andamento per le varie province. Ho anche i dati di google per ogni provincia volendo. Posso fare un discorso per ogni provincia, poi per tutta l'italia insieme (media dei dati) e infine per un altro stato volendo.

Potrei fare un indice di correlazione tra questi e Rt.

In [134]:
average_network_degree = pd.read_csv('data/italy_mobility/average_network_degree_2020_01_18_2020_06_26.csv')
regioni = pd.read_csv('data/italy_mobility/province-italiane.csv', delimiter=';')
regioni = regioni.drop(columns=['Provincia'])
prova = pd.merge(average_network_degree, regioni, how='inner', left_on = 'SIGLA', right_on = 'Sigla')
prova = prova.drop(columns=['Sigla', 'COD_PROV', 'SIGLA', 'Unnamed: 0', 'NAME_PROV'])
prova = prova.groupby('Regione', as_index=False, sort=False)[prova.columns[:-47]].mean().T
new_header = np.array(prova.iloc[0]).astype(str)
new_header[3] = 'Aosta'
new_header[9] = 'Emilia_romagna'
new_header[10] = 'Trentino_alto_adige'
new_header[17] = 'Friuli_venezia_giulia'
#new_header = np.insert(new_header, 0,'date')
prova = prova[1:]
#prova = prova.reset_index(level=0)
prova.columns = new_header #set the header row as the df header
prova

,Sicilia,Piemonte,Marche,Aosta,Toscana,Campania,Puglia,Veneto,Lombardia,Emilia_romagna,Trentino_alto_adige,Sardegna,Molise,Calabria,Abruzzo,Lazio,Liguria,Friuli_venezia_giulia,Basilicata,Umbria
2020-01-18,0.032607,0.070314,0.039345,0.060936,0.044176,0.034421,0.048846,0.061484,0.068396,0.054328,0.052035,0.030614,0.040531,0.015009,0.026493,0.015781,0.133265,0.07011,0.033453,0.061778
2020-01-19,0.032743,0.062471,0.027711,0.06836,0.034618,0.027277,0.051333,0.073318,0.06216,0.064746,0.038917,0.018993,0.026211,0.016148,0.024737,0.0147,0.092945,0.048396,0.029962,0.035375
2020-01-20,0.040526,0.065758,0.02668,0.087429,0.034348,0.035067,0.047946,0.040164,0.057071,0.057425,0.047449,0.025217,0.045224,0.03049,0.027448,0.017213,0.111328,0.051808,0.027992,0.046087
2020-01-21,0.044176,0.071704,0.029802,0.099544,0.04277,0.037868,0.053326,0.043134,0.059798,0.056183,0.058019,0.022042,0.03452,0.027941,0.025639,0.02136,0.11258,0.059412,0.022699,0.04459
2020-01-22,0.039271,0.068018,0.025704,0.073563,0.039657,0.03573,0.049455,0.043082,0.061403,0.053375,0.050915,0.025619,0.034017,0.026764,0.021297,0.023937,0.117182,0.070226,0.027663,0.032437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-07,0.022024,0.035343,0.022748,0.051915,0.026653,0.019877,0.029637,0.02607,0.031705,0.027294,0.030181,0.014118,0.020627,0.017661,0.014853,0.012025,0.057129,0.033171,0.014358,0.030461
2020-05-08,0.027768,0.038257,0.021644,0.055795,0.029324,0.022938,0.030904,0.026693,0.034085,0.029858,0.030036,0.014549,0.014061,0.015778,0.016431,0.016013,0.064758,0.03617,0.028217,0.031129
2020-05-09,0.020683,0.035794,0.01769,0.039145,0.026242,0.022558,0.038723,0.025246,0.032234,0.027842,0.03145,0.021461,0.015939,0.016491,0.015943,0.017367,0.052258,0.035281,0.016859,0.043916
2020-05-10,0.015015,0.02514,0.020636,0.028939,0.020035,0.018112,0.025476,0.016837,0.026822,0.021186,0.022376,0.011792,0.013048,0.01535,0.012097,0.008309,0.044585,0.020617,0.01651,0.024531


In [135]:
df = pd.read_csv('data/Region_Mobility_Report_CSVs/2020_IT_Region_Mobility_Report.csv')
df = df[~df['sub_region_1'].isnull()]
region_names = pd.unique(df['sub_region_1'])
region_names_italian = ['abruzzo', 'aosta','piemonte','basilicata','calabria','campania','emilia_romagna','friuli_venezia_giulia',
'lazio','liguria','lombardia','marche','molise','piemonte','sardegna','sicilia','trentino_alto_adige','toscana','umbria','veneto']
# Dataframe dove storare i risultati
region_report = pd.DataFrame(region_names, columns = ['region_name'])
xdata_covid = xdata
names = ['retail_and_recreation_percent_change_from_baseline',
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline',
        'residential_percent_change_from_baseline']

NameError: name 'xdata' is not defined

File con per ogni riga una provincia, e in colonna il radius of gyration. Questo è indice di quanto si sposta in media una persona. come varia da prima e dopo lockdown? E' correlato con < k >?

. Potrei fare un indice di correlazione tra questi e Rt medio di ogni settimana.

In [19]:
median_q1_q3 = pd.read_csv('data/italy_mobility/median_q1_q3_rog_2020_01_18_2020_06_26.csv')

In [23]:
regioni = pd.read_csv('data/italy_mobility/province-italiane.csv', delimiter=';')
regioni = regioni.drop(columns=['Provincia'])

In [55]:
prova = pd.merge(median_q1_q3, regioni, how='inner', left_on = 'SIGLA', right_on = 'Sigla')
prova = prova.drop(columns=['Sigla', 'COD_PROV', 'DEN_PCM', 'SIGLA'])
prova.columns[:-47]

Index(['18/01-24/01', '25/01-31/01', '01/02-07/02', '08/02-14/02',
       '15/02-21-02', '22/02-28/02', '29/02-06/03', '07/03-13/03',
       '14/03-20/03', '21/03-27/03', '28-03/03-04', '04-04/10-04',
       '11/04-17/04', '18/04-24/04', '25/04-01/05', '02/05-08/05',
       '09/05-15/05', '16/05-22/05', '23/05-29/05', '30/05-05/06',
       '06/06-12/06', '13/06-19/06', '20/06-26/06'],
      dtype='object')

In [96]:
prova = pd.merge(median_q1_q3, regioni, how='inner', left_on = 'SIGLA', right_on = 'Sigla')
prova = prova.drop(columns=['Sigla', 'COD_PROV', 'DEN_PCM', 'SIGLA'])
prova = prova.groupby('Regione', as_index=False, sort=False)[prova.columns[:-47]].mean().T
new_header = np.array(prova.iloc[0]).astype(str)
new_header = np.insert(new_header, 0,'date')
#new_header = prova.iloc[0] #grab the first row for the header
# new_header
prova = prova[1:] #take the data less the header row
#prova['date'] = prova.index
prova = prova.reset_index(level=0)
prova.columns = new_header #set the header row as the df header
prova

,date,Sicilia,Piemonte,Marche,Valle d'Aosta,Toscana,Campania,Puglia,Veneto,Lombardia,...,Trentino Alto Adige,Sardegna,Molise,Calabria,Abruzzo,Lazio,Liguria,Friuli Venezia Giulia,Basilicata,Umbria
0,18/01-24/01,2.69104,3.55951,3.685621,3.173583,3.238725,2.794326,2.679236,3.680627,3.207578,...,3.239778,3.038615,3.338424,2.893445,3.162265,4.796038,2.278965,3.311548,4.370706,2.73328
1,25/01-31/01,2.459693,3.630308,3.416765,3.442209,3.438736,2.846682,2.793829,3.700066,3.178596,...,3.359123,2.993896,3.125336,2.484693,3.169635,4.157352,2.186154,3.21486,3.148455,3.374712
2,01/02-07/02,2.763466,3.640555,3.673329,3.133672,3.291643,3.034018,2.83494,3.779488,3.295481,...,3.477022,3.116953,3.349679,2.660518,3.343424,4.347598,2.349306,3.230968,3.572514,2.939544
3,08/02-14/02,2.461909,3.645121,3.603822,3.064928,3.570646,2.941594,2.720196,3.864038,3.241577,...,3.341928,3.112491,3.221192,2.728988,3.251592,4.323309,2.360374,3.262694,3.84542,3.059054
4,15/02-21-02,2.51744,3.563353,3.744869,3.113111,3.508768,2.893794,2.783981,3.954026,3.28638,...,3.671891,3.306966,3.094988,2.632932,3.057246,4.396049,2.535134,3.346083,3.651293,2.999741
5,22/02-28/02,2.528952,3.285577,3.453863,2.804597,3.426423,2.706394,2.650459,3.351687,2.613556,...,3.272478,3.092828,2.873255,2.2207,3.156637,4.263579,2.161895,2.903402,3.272619,2.993651
6,29/02-06/03,2.374005,3.078722,2.994751,2.813787,2.974412,2.508139,2.566491,3.195133,2.315502,...,2.955619,2.787099,2.67033,2.392134,3.460377,4.211882,1.888039,2.480936,2.939639,2.608926
7,07/03-13/03,1.317162,1.856893,1.807469,1.830811,1.765116,1.404538,1.139859,2.251133,1.299303,...,1.750017,1.569883,1.295476,1.498719,1.758817,2.565832,1.214157,1.847364,2.207473,1.631617
8,14/03-20/03,0.490122,0.607827,0.588394,0.707621,0.498719,0.527848,0.38864,0.55495,0.414373,...,0.417955,0.570161,0.669074,0.550723,0.725157,0.915227,0.347691,0.462223,0.824757,0.81759
9,21/03-27/03,0.397315,0.453122,0.480681,0.468915,0.37993,0.469258,0.325261,0.436573,0.298849,...,0.34795,0.608432,0.689055,0.479548,0.489593,0.788381,0.264495,0.342893,0.720139,0.541713
